In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import json
import random
import fastwer
from tqdm.auto import tqdm

In [14]:
CKP_MODEL = "qantev/trocr-large-spanish" #"/home/ralvarez22/Documentos/llm_data/llm_cache/models--microsoft--trocr-large-stage1/snapshots/3c8ead8dfda428d914334169380bb546f770a300"
DEVICE = "cuda" # Where to load the model
DATASET_FILE = "/home/ralvarez22/Documentos/trocr_hand/trocr_llm/datasets/cursive_hand_cropped/metadata.json"

In [15]:
processor = TrOCRProcessor.from_pretrained(CKP_MODEL,device_map=DEVICE)
model = VisionEncoderDecoderModel.from_pretrained(CKP_MODEL, device_map=DEVICE)
# This configuration allows to correctly set the BOS token for inference and the Temperature and Sample strategy for the decoder
# This could be setted on the generation_config.json file, generated when saving the model, but, to avoid mismatch or errors, I overwrite this
model.generation_config.decoder_start_token_id = processor.tokenizer.bos_token_id
model.generation_config.temperature = 0.4
model.generation_config.do_sample = True

preprocessor_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.97k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/420 [00:00<?, ?B/s]

In [7]:
cropped_images_dataset = json.load(open(DATASET_FILE, "r"))

In [8]:
items_to_eval = int(len(cropped_images_dataset) * 0.9)

In [9]:
items_to_eval

3472

In [10]:
test_dataset = random.sample(cropped_images_dataset, items_to_eval)

In [16]:
cer_value = 0
wer_value = 0
for test_item in tqdm(test_dataset):
    pil_image = Image.open(test_item["image"]).convert("RGB")
    proc_image = processor(pil_image, return_tensors="pt").pixel_values.to(DEVICE)
    tgt_text = test_item["label"]
    data_generated = model.generate(proc_image)
    gen_text = processor.tokenizer.decode(data_generated[0].cpu(), skip_special_tokens=True)
    #print("Original: {} - Recognized: {}".format(tgt_text, gen_text))
    cer_val = fastwer.score_sent(gen_text, tgt_text, char_level=True)
    wer_val = fastwer.score_sent(gen_text, tgt_text)
    cer_value += cer_val
    wer_value += wer_val

  0%|          | 0/3472 [00:00<?, ?it/s]

In [17]:
cer_value /= items_to_eval
wer_value /= items_to_eval

In [18]:
print("CER Avg value: {:.4f}".format(cer_value))
print("WER Avg value: {:.4f}".format(wer_value))

CER Avg value: 69.6941
WER Avg value: 99.5080
